In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import numpy as np 
import requests
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor

Scraping papers of 2023 and 2024 from the archive site

In [ ]:
aaai_papers = {
        "Year" : [], 
        "Title" : [], 
        "Authors" : [],
        "Number of authors" : [],
        "Abstract" : [],
        "Citations" : [],
        "Keyword" : [],
        "Paper file" : []
        
}

original_url = "https://ojs.aaai.org/index.php/AAAI/issue/archive"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
content = requests.get(original_url,headers=headers).text

scraper = BeautifulSoup(content, parser="html.parser")

tracks = scraper.find("ul", class_="issues_archive")

track_links = tracks.find_all("li")
i=0

for  li in track_links:
    papers_link = li.find('h2').find('a').get("href")

    track_title = li.find('h2').find('a').text

    if "24" in track_title:
        year = "2024"
    elif "23" in track_title:
        year ="2023"
    else:
        break
    
    papers_page = requests.get(papers_link,headers=headers).text

    scraper = BeautifulSoup(papers_page, parser="html.parser")

    div_tracks = scraper.find('div', class_='sections')
    div_sections = scraper.find_all('div', class_='section')

    for section in div_sections :
        keyword = section.find('h2').text
        
        papers = section.find('ul').find_all('li', recursive=False)
        for paper in papers:

            print("year: ",year," ",i)
            i=i+1

            paper_title=paper.find('h3').find('a').text
           
            paper_link=paper.find('h3').find('a').get("href")
            authors=paper.find('div', class_="authors").text
            authors = authors.lstrip()
            pdf_link = paper.find_all('li')[0].find('a').get("href")

            paper_page = requests.get(paper_link,headers=headers).text
            scraper = BeautifulSoup(paper_page, parser="html.parser")

            abstract= scraper.find('section', class_='item abstract')
            abstract.find('h2').extract()
            abstract = abstract.get_text(strip=True)
            aaai_papers["Year"].append(year)
            aaai_papers["Title"].append(paper_title.lstrip())
            aaai_papers["Authors"].append(authors)
            aaai_papers["Number of authors"].append(len(authors.split(",")))
            aaai_papers["Abstract"].append(abstract)
            aaai_papers["Citations"].append(0)  
            aaai_papers["Paper file"].append(pdf_link.lstrip())
            aaai_papers["Keyword"].append(keyword.lstrip()) 

papers_23_24 = pd.DataFrame(aaai_papers)

Scraping papers of years from 2022 to 1980

In [ ]:
##scraping one year
def scrape_year(a):
    
    aaai_papers = {
        "Year" : [], 
        "Title" : [], 
        "Authors" : [],
        "Number of authors" : [],
        "Abstract" : [],
        "Citations" : [],
        "Keyword" : [],
        "Paper file" : []
        
    }
    
    i=0
    year = a.text
    year_link = a.get('href')
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    content_year_page = requests.get(year_link,headers=headers).text
    scraper = BeautifulSoup(content_year_page, parser="html.parser")

    div_element = scraper.find('div', class_='content-sidebar-wrap')
    li_elements = div_element.find_all('li')
    for li in li_elements:
        a_elements = li.find('a')
        track_link = a_elements.get('href')
        track_page = requests.get(track_link,headers=headers).text
        scraper = BeautifulSoup(track_page, parser="html.parser")

        div_tracks = scraper.find_all('div', class_='track-wrap')
        
        for div_track in div_tracks:
            keyword = div_track.find('h2').text
            li_elements = div_track.find_all('li')
            for li in li_elements:
                paper_link = li.find('h5').find('a').get('href')
                title = li.find('h5').find('a').text
                authors = li.find('span').find('p').text
                pdf_link = li.find('a', class_='wp-block-button').get('href')
                paper_page = requests.get(paper_link,headers=headers).text
                scraper = BeautifulSoup(paper_page, parser="html.parser")

                div_paper= scraper.find_all('div', class_='paper-section-wrap')
                for div in div_paper:
                    if (div.find('div', class_='attribute-output') is not None):
                        abstract = div.find('div', class_='attribute-output').text
                    break
                
                
                if (div.find('div', class_='attribute-output') is None) :
                    abstract = " "
                    print(" ************ none ********************")

                aaai_papers["Year"].append(year)
                aaai_papers["Title"].append(title)
                aaai_papers["Authors"].append(authors)
                aaai_papers["Number of authors"].append(len(authors.split(",")))
                aaai_papers["Abstract"].append(abstract)
                aaai_papers["Citations"].append(0)  
                aaai_papers["Paper file"].append(pdf_link)
                aaai_papers["Keyword"].append(keyword)   
                print("Year: ",year, "paper: ", i ) 
                i=i+1      
    return pd.DataFrame(aaai_papers)


## scraping 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
original_url = "https://aaai.org/aaai-publications/aaai-conference-proceedings"
content_years_page = requests.get(original_url,headers=headers).text
scraper = BeautifulSoup(content_years_page, parser="html.parser")


p_element = scraper.find('p', class_='link-block has-brand-color-7-color has-text-color')

a_elements = p_element.find_all('a')

with ThreadPoolExecutor(max_workers=7) as executor:
    results = list(executor.map(scrape_year, a_elements[2:]))



Saving results into a csv file

In [ ]:
#concat all the results
papers = pd.DataFrame()

papers = pd.concat(papers_23_24)
papers = pd.concat(results)

if not os.path.isdir("folder_to_save"):
    os.makedirs("folder_to_save")

csv_file = os.path.join("folder_to_save", "aaai_papers.csv")
papers.to_csv(csv_file, index=False)